In [ ]:
productFiles = [
    'm_knitwears.txt',
    'w_knitwears.txt'
]

In [ ]:
from selenium.webdriver.firefox.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.common.keys import Keys
import re
import os
import time
import requests
import sys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from urllib.parse import urlparse
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import multiprocessing
import requests
import urllib
from urllib.parse import urlparse

In [ ]:
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)
driver.maximize_window()
# driver.get("https://www.bonprix.co.uk/products/all-weather-winter-jacket/_/A-953107_44R?PFM_rsn=browse&PFM_ref=false&PFM_psp=own")

In [ ]:
def extract_links(link):
    driver.get("https://www.bonprix.co.uk/products/all-weather-winter-jacket/_/A-953107_44R?PFM_rsn=browse&PFM_ref=false&PFM_psp=own")
    SCROLL_PAUSE_TIME = 1    
#   Infinite scroll handling
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    imgs = driver.find_elements_by_css_selector('div.altProductImagesContainer > ul > li > img')
    print(imgs)
    for img in imgs:
        src = img.get_attribute('data-original')
        print(src)

In [ ]:
def extract_link(file):
    os.chdir('/home/uplarauser/bon/')
    print(os.getcwd())
    f = open(file,'r')
    temp = []
    for line in f:
        line = line.replace('\n','')
        temp.append(line)
    print(len(temp))
    temp = list(set(temp))
    print(len(temp))
    curr_dir = os.getcwd() + '/'
    os.chdir(curr_dir + file.replace('.txt','') + '/bonprix/')
    main_dir = os.getcwd()
    options = Options()
    options.headless = True
    driver = webdriver.Firefox(options=options)
    driver.maximize_window()
    i = 0
    for link in temp:
        driver.get(link + '?PFM_rsn=browse&PFM_ref=false&PFM_psp=own')
        SCROLL_PAUSE_TIME = 1    
#   Infinite scroll handling
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        imgs = driver.find_elements_by_css_selector('div.altProductImagesContainer > ul > li > img')
        srcs = []
        for img in imgs:
            src = img.get_attribute('data-image')
            if src==None:
                src = img.get_attribute('data-original')
            srcs.append(src)
        srcs = list(set(srcs))
        product_colors = {}
        for imgsrc in srcs:
            imgsrc = imgsrc.split('/')
            imgsrc[6] = '466w'
            product_name = imgsrc[7]
            newimgsrc = '/'.join(imgsrc)
            print(newimgsrc)
            product_name = product_name.replace('.jpg','')
            product_color = product_name.split('~')[1].split('_')[0][0:6]
            if product_color in product_colors:
                product_colors[product_color].append(newimgsrc)
            else:
                product_colors[product_color] = [newimgsrc]
        for key,value in product_colors.items():
            print(key,'here')
            flag = 0
            for product in value:
                try:
                    if flag==0:
                        folder_name = product.split('/')[-1].replace('.jpg','')
                        os.system("mkdir "+ folder_name)
                        os.chdir(os.getcwd() + '/' + folder_name)
                    flag+=1
                    try:
                        urllib.request.urlretrieve(product, str(flag) + ".jpg")
                    except:
                        pass
                except:
                    pass
            os.chdir(main_dir)
        i+=1
        print(i,'ends')
    driver.quit()

In [ ]:
for file in productFiles:
    print(file)
    extract_link(file)

In [ ]:
driver.quit()